In [122]:
import os
import sys
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from random import shuffle

In [5]:
DATA_PATH = '../TlkPersonaChatRus/'

## Dialogs

In [6]:
dialogs = pd.read_csv(os.path.join(DATA_PATH, 'dialogues.tsv'), sep='\t')

In [7]:
dialogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10013 entries, 0 to 10012
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   persona_1_profile  10013 non-null  object
 1   persona_2_profile  10013 non-null  object
 2   dialogue           10013 non-null  object
dtypes: object(3)
memory usage: 234.8+ KB


### dialogs

In [35]:
some_text = dialogs['dialogue'][1]
some_text

'<span class=participant_1>Пользователь 1: Привет!</span><br /><span class=participant_2>Пользователь 2: Привет,Как жизнь?</span><br /><span class=participant_1>Пользователь 1: Отлично) Солнышко светит, птички поют!</span><br /><span class=participant_2>Пользователь 2: Я вот сегодня понял, что меня тупо используют, всем<br />нужны от меня лишь деньги, ненавижу людей</span><br /><span class=participant_2>Пользователь 2: Чем занимаешься по жизни, я вот бизнесмен.</span><br /><span class=participant_1>Пользователь 1: А я вот учу детей, работаю с начальными классами</span><br /><span class=participant_1>Пользователь 1: Не все люди такие, как ты говоришь</span><br /><span class=participant_1>Пользователь 1: Помимо работы чем еще ты занимаешься?</span><br /><span class=participant_2>Пользователь 2: К свадьбе готовлюсь</span><br /><span class=participant_2>Пользователь 2: А ты?</span><br /><span class=participant_1>Пользователь 1: Вот видишь) значит, нашел такую женщину, которой не<br />нужны

some replicas are corrupted by `<br />` symbol - need to fix this

In [39]:
def print_dialog(row):
    text = row['dialogue'].replace('<br />', ' ')
    print('\n'.join(t.text for t in BeautifulSoup(text, features='html').find_all('span')))
    print('\n' + '#'*80 + '\n')

In [51]:
for i, cols in dialogs.iterrows():
    if i == 20:
        break
    print_dialog(cols)

Пользователь 2: Привет) расскажи о себе
Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 2: Что читаешь? Мне нравится классика
Пользователь 2: Я тоже люблю пообщаться
Пользователь 1: Люблю животных, просто обожаю, как и свою работу)
Пользователь 1: Я фантастику люблю
Пользователь 2: А я выращиваю фиалки
Пользователь 2: И веду здоровый и активный образ жизни!
Пользователь 1: Ух ты, интересно.
Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду ..
Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим другом)

################################################################################

Пользователь 1: Привет!
Пользователь 2: Привет,Как жизнь?
Пользователь 1: Отлично) Солнышко светит, птички поют!
Пользователь 2: Я вот сегодня понял, что меня тупо используют, всем нужны от меня лишь деньги, ненавижу людей
Пользователь 2: Чем занимаешься по жизни, я вот бизнесмен.
Пользователь 1: А я вот учу детей, работаю с начальн

as in usual dialog there can be some consequtive replicas from one user

In [54]:
dialogs.persona_1_profile[0], dialogs.persona_2_profile[0], 

('<span class=participant_1>У меня любимая работа.<br />Я уважаю людей.<br />У меня есть животное.<br />У меня хороший друг.<br />Я люблю кофе.<br /></span>',
 '<span class=participant_2>Ищу принца.<br />Веду активный образ жизни.<br />Люблю читать классику.<br />Выращиваю фиалки.<br />Люблю общение.<br /></span>')

In [56]:
pers = dialogs.persona_1_profile[1]

In [63]:
dialogs[(dialogs['persona_1_profile'] == pers) | (dialogs['persona_2_profile'] == pers)]

,persona_1_profile,persona_2_profile,dialogue
1,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />У м...,<span class=participant_1>Пользователь 1: Прив...
1852,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я завуч.<br />У меня...,<span class=participant_2>Пользователь 2: Прив...
3871,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Веселая девушка.<br ...,"""<span class=participant_2>Пользователь 2: При..."
7558,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я Саша<br />У меня е...,"""<span class=participant_2>Пользователь 2: При..."
9310,<span class=participant_1>Я работаю учителем<b...,<span class=participant_2>Я бизнесмен<br />Я л...,<span class=participant_2>Пользователь 2: прив...


# IDEA - combine one type of personality and train on this

## Profiles

In [48]:
profiles = pd.read_csv(os.path.join(DATA_PATH, 'profiles.tsv'), sep='\t', )

In [49]:
profiles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1505 entries, У меня любимая работа. to Я кулинар.
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   characteristic_1  1505 non-null   object 
 1   characteristic_2  1505 non-null   object 
 2   characteristic_3  1505 non-null   object 
 3   characteristic_4  1505 non-null   object 
 4   characteristic_5  0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 70.5+ KB


In [50]:
profiles.head(20)

,characteristic_1,characteristic_2,characteristic_3,characteristic_4,characteristic_5
У меня любимая работа.,Я уважаю людей.,У меня есть животное.,У меня хороший друг.,Я люблю кофе.,NaN
Я работаю учителем,У меня есть собака,Я люблю петь,Я живу сама,Я люблю цветы,NaN
Я купила дом,Я бегаю по утрам,Я работаю на работе,Я поеду в отпуск,Я люблю арбуз,NaN
я врач и женат,у меня трое детей,не люблю свою работу,нравиться ездить на велосипеде,люблю пиво,NaN
Я школьница.,Я ещё учусь.,Но я мечтаю работать.,Я обожаю родителей.,И не люблю учиться.,NaN
Я дизайнер.,Я был в 12-ти странах.,Люблю пешие прогулки.,люблю рок-музыку.,Хочу купить дачу.,NaN
я норвежец,у меня есть попугай,я не люблю деревья,у меня было десять лошадей,я не курю,NaN
Я автослесарь,Люблю играть на гитаре,Обожаю рыбалку,Воспитываю дочку,Вдовец,NaN
Я рисую,Я живу за границей,Я посадила семь яблонь,Я работаю с дельфинами,Я варю вкусные щи,NaN
Я красивая,У меня светлые волосы,Живу на первом этаже,Я очень люблю животных,У меня была собака,NaN


In [65]:
os.listdir(DATA_PATH)

['valid.txt',
 'train.txt',
 'new_dialogues.tsv',
 'profiles.tsv',
 'pretrain_transformers.py',
 'readme_TlkPersonaChatRus.txt',
 'dialogues.tsv']

# New dialogs

In [69]:
new_dialogs = pd.read_csv(os.path.join(DATA_PATH, 'new_dialogues.tsv'), sep='\t', index_col=0)

In [84]:
new_dialogs

,dialogue,persona_1_profile_1,persona_1_profile_2,persona_1_profile_3,persona_1_profile_4,persona_1_profile_5,persona_2_profile_1,persona_2_profile_2,persona_2_profile_3,persona_2_profile_4,persona_2_profile_5,persona_1
0,<span class=participant_2>Пользователь 2: Прив...,любимая работа,уважаю людей,есть животное,хороший друг,люблю кофе,ищу принца,веду активный образ жизни,люблю читать классику,выращиваю фиалки,люблю общение,любимая работа уважаю людей есть животное хоро...
1,<span class=participant_1>Пользователь 1: Прив...,работаю учителем,есть собака,люблю петь,живу сама,люблю цветы,бизнесмен,скоро свадьба,меня любят только за деньги,не люблю людей,не люблю тупые опросы,работаю учителем есть собака люблю петь живу с...
2,<span class=participant_1>Пользователь 1: Прив...,купила дом,бегаю по утрам,работаю на работе,поеду в отпуск,люблю арбуз,пою в караоке,есть супруга,хорошо готовлю пасту,люблю собак,аллергия на кошек,купила дом бегаю по утрам работаю на работе по...
3,<span class=participant_2>Пользователь 2: Здра...,врач и женат,трое детей,не люблю свою работу,нравиться ездить на велосипеде,люблю пиво,мальчик,учусь в 6-ом классе,хочу стать гонщиком,есть сестра,мечтаю о машине,врач и женат трое детей не люблю свою работу н...
4,<span class=participant_1>Пользователь 1: Прив...,школьница,ещё учусь,но я мечтаю работать,обожаю родителей,и не люблю учиться,простоват,люди избегают меня,быстро бегаю,мои увлечения неординарны,работаю по призванию,школьница ещё учусь но я мечтаю работать обожа...
...,...,...,...,...,...,...,...,...,...,...,...,...
10008,"""<span class=participant_1>Пользователь 1: При...",6 собак,не люблю кошек,работаю кинологом,люблю музыку,не люблю гулять,музыкант,люблю веселые компании,увлекаюсь охотой,люблю мотоциклы,люблю собирать грибы,6 собак не люблю кошек работаю кинологом люблю...
10009,<span class=participant_1>Пользователь 1: Прив...,верная,мне нравится уют,люблю посиделки у камина,мое хобби вязание,не люблю дождь,люблю рок,пишу статьи для журнала,езжу на велосипеде,хочу купить собаку,не люблю морковку,верная мне нравится уют люблю посиделки у кам...
10010,<span class=participant_1>Пользователь 1: Прив...,студент,учусь на хирурга,очень люблю комедии,люблю машины,есть золотая рыбка,директор турфирмы,люблю детей,катаюсь на сноуборде,люблю звонить по ночам,хочу покорить эверест,студент учусь на хирурга очень люблю комедии л...
10011,<span class=participant_1>Пользователь 1: Прив...,моя мама живет со мной,люблю есть пироги,есть хаски,часто болею,рисую животных,воспитатель,люблю готовить,люблю велоспорт,увлекаюсь магией,есть черная кошка,моя мама живет со мной люблю есть пироги есть ...


In [89]:
from itertools import chain

new_dialogs['persona_1'] = new_dialogs[[f'persona_1_profile_{i}' for i in range(1,6)]].values.tolist()
new_dialogs['persona_2'] = new_dialogs[[f'persona_2_profile_{i}' for i in range(1,6)]].values.tolist()

In [90]:
new_dialogs['persona_2'][0]

['ищу принца',
 'веду активный образ жизни',
 'люблю читать классику',
 'выращиваю фиалки',
 'люблю общение']

In [93]:
def preprocess_for_raw_text(dialog):
    dialog = dialog.replace('<br />', ' ').replace('</span> ', '')
    dialog = dialog.replace('<span class=participant_1>Пользователь 1: ', '<speaker1>')
    dialog = dialog.replace('<span class=participant_2>Пользователь 2: ', '<speaker2>')
    return dialog

In [94]:
new_dialogs['dialogue_preprocessed'] = new_dialogs['dialogue'].apply(preprocess_for_raw_text)

In [95]:
new_dialogs[['dialogue_preprocessed', 'persona_1', 'persona_2']]

,dialogue_preprocessed,persona_1,persona_2
0,<speaker2>Привет) расскажи о себе<speaker1>При...,"[любимая работа, уважаю людей, есть животное, ...","[ищу принца, веду активный образ жизни, люблю ..."
1,"<speaker1>Привет!<speaker2>Привет,Как жизнь?<s...","[работаю учителем, есть собака, люблю петь, жи...","[бизнесмен, скоро свадьба, меня любят только з..."
2,<speaker1>Привет<speaker1>Как дела ?<speaker2>...,"[купила дом, бегаю по утрам, работаю на работе...","[пою в караоке, есть супруга, хорошо готовлю п..."
3,<speaker2>Здравствуйте<speaker2>Я Леша<speaker...,"[врач и женат, трое детей, не люблю свою работ...","[мальчик, учусь в 6-ом классе, хочу стать гонщ..."
4,<speaker1>Привет!<speaker2>Привет!<speaker2>Ка...,"[школьница, ещё учусь, но я мечтаю работать, о...","[простоват, люди избегают меня, быстро бегаю, ..."
...,...,...,...
10008,"""<speaker1>Привет<speaker2>Привет, как настрое...","[6 собак, не люблю кошек, работаю кинологом, л...","[музыкант, люблю веселые компании, увлекаюсь о..."
10009,<speaker1>Привет)<speaker1>Как жизнь? Чем увле...,"[верная, мне нравится уют , люблю посиделки у ...","[люблю рок, пишу статьи для журнала, езжу на в..."
10010,<speaker1>Привет<speaker2>приветствую<speaker1...,"[студент, учусь на хирурга, очень люблю комеди...","[директор турфирмы, люблю детей, катаюсь на сн..."
10011,<speaker1>Приветик)<speaker2>Привет. Меня зову...,"[моя мама живет со мной, люблю есть пироги, ес...","[воспитатель, люблю готовить, люблю велоспорт,..."


In [237]:
def form_raw_text(dialog):
    res = []
    prev_speaker = dialog[:9]
    for ind, char in enumerate(dialog):
        if char == '<' and dialog[ind:ind + 9] != prev_speaker:
            res.append('\n')
            prev_speaker = ''.join(dialog[ind:ind + 9])
        res.append(char)
    return ''.join(res).split('\n')

In [239]:
dialog = form_raw_text(new_dialogs['dialogue_preprocessed'][0])

result = list()

for i in range(2, len(dialog)):
    if i <= 4:
        result.append(dialog[:i])
    else:
        result.append(dialog[i - 4: i])
result

[['<speaker2>Привет) расскажи о себе',
  '<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )'],
 ['<speaker2>Привет) расскажи о себе',
  '<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )',
  '<speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться'],
 ['<speaker2>Привет) расскажи о себе',
  '<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )',
  '<speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться',
  '<speaker1>Люблю животных, просто обожаю, как и свою работу)<speaker1>Я фантастику люблю'],
 ['<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )',
  '<speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться',
  '<speaker1>Люблю животных, просто обожаю, как и свою работу)<speaker1>Я фантастику люблю',
  '<speaker2>А я выращиваю фиалки<speaker2>И веду здоровый и активный образ жизни!'],
 ['<speaker2>Что читаешь? Мне нравится классика<speaker2>Я 

In [225]:
def get_chunks(dlg, histoty_size=2):
    result = list()

    for i in range(histoty_size, len(dlg) + 1):
        if i <= histoty_size * 2:
            result.append(dlg[:i])
        else:
            result.append(dlg[i - histoty_size * 2: i])
    return result

In [285]:
def iclude_all_in_dialog(history_and_replica, bos_token, eos_token, include_persona=False, persona_1=None, persona_2=None):
    result = list()
    for replica in history_and_replica:
        if include_persona:
            if replica[-1][:10] == '<speaker1>':
                shuffle(persona_1)
                person = persona_1
            else:
                shuffle(persona_2)
                person = persona_2
            result.append(' '.join(chain([bos_token] + ['<person>'] + person + replica + [eos_token])))
        else:
            result.append(' '.join([bos_token] + replica + [eos_token]))
    return result

In [286]:
iclude_all_in_dialog(get_chunks(dialog), '<bos>', '<eos>', True,
                     ['люблю животных', 'не женат', 'люблю свою работу'],
                     ['люблю читать', 'девушка', 'люблю цветы'])

['<bos> <person> люблю свою работу не женат люблю животных <speaker2>Привет) расскажи о себе <speaker1>Привет) под вкусный кофеек настроение поболтать появилось ) <eos>',
 '<bos> <person> девушка люблю цветы люблю читать <speaker2>Привет) расскажи о себе <speaker1>Привет) под вкусный кофеек настроение поболтать появилось ) <speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться <eos>',
 '<bos> <person> не женат люблю животных люблю свою работу <speaker2>Привет) расскажи о себе <speaker1>Привет) под вкусный кофеек настроение поболтать появилось ) <speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться <speaker1>Люблю животных, просто обожаю, как и свою работу)<speaker1>Я фантастику люблю <eos>',
 '<bos> <person> девушка люблю читать люблю цветы <speaker1>Привет) под вкусный кофеек настроение поболтать появилось ) <speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться <speaker1>Люблю животных, просто обожаю, как и свою ра

In [287]:
from tqdm.notebook import tqdm


def form_data_to_train(dialogs, bos_token, eos_token, include_persona=False, persona_1=None, persona_2=None):
    result = []
    if include_persona:
        iterator = zip(dialogs, persona_1, persona_2)
    else:
        iterator = dialogs
    for chunk in tqdm(iterator):
        if include_persona:
            dialog, per_1, per_2 = chunk
            replicas = iclude_all_in_dialog(get_chunks(form_raw_text(dialog)),
                                        '<bos>', '<eos>',
                                        include_persona, per_1, per_2)
        else:
            dialog = chunk
            replicas = iclude_all_in_dialog(get_chunks(form_raw_text(dialog)), '<bos>', '<eos>')
        result.extend(replicas)
    return result

In [288]:
dialogs = new_dialogs['dialogue_preprocessed'][:5]
pers_1 = new_dialogs['persona_1'][:5]
pers_2 = new_dialogs['persona_2'][:5]
form_data_to_train(dialogs, '<bos>', '<eos>', False, pers_1, pers_2);
# form_raw_text(new_dialogs['dialogue_preprocessed'][:5][0]).split('\n')

  0%|          | 0/5 [00:00<?, ?it/s]

In [289]:
res = form_data_to_train(new_dialogs['dialogue_preprocessed'], '<bos>', '<eos>',
                   True,
                   new_dialogs['persona_1'],
                   new_dialogs['persona_2'])

0it [00:00, ?it/s]

In [278]:
shuffle(res)

train = res[:int(len(res) * 0.85)]
valid = res[int(len(res) * 0.85):]

with open(os.path.join(DATA_PATH, 'train_hist.txt'), 'w') as f:
    f.write('\n'.join(train))

with open(os.path.join(DATA_PATH, 'valid_hist.txt'), 'w') as f:
    f.write('\n'.join(valid))

In [291]:
shuffle(res)

train = res[:int(len(res) * 0.85)]
valid = res[int(len(res) * 0.85):]

with open(os.path.join(DATA_PATH, 'train_pers+hist.txt'), 'w') as f:
    f.write('\n'.join(train))

with open(os.path.join(DATA_PATH, 'valid_pers+hist.txt'), 'w') as f:
    f.write('\n'.join(valid))